In [1]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
from utils.loader import PairSetMNIST
from torch.utils.data import Dataset, DataLoader

In [2]:
# load the dataset as a Dataset object
train_data = PairSetMNIST(train=True)
test_data  = PairSetMNIST(test=True)

In [3]:
# architectures 

class LeNet_WS_sequential(nn.Module):
    """
    Weight sharing 
    
    """
    def __init__(self, nb_hidden):
        super(LeNet_WS_sequential, self).__init__()
        
        # convolutional weights for digit reocgnition shared for each image
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)
        # fully connected layers 
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):        
        
        # split the 2-channel input into two 14*14 images
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        # forward pass for the first image 
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        # forward pass for the second image
        y = F.relu(F.max_pool2d(self.conv1(y), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size=2, stride=2))
        y = F.relu(self.fc1(y.view(-1, 256)))
        y = self.fc2(y)
        
        # concatenate layers 
        z = torch.cat([x, y], 1)
        
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = self.fc5(z)
        
        return  z

    


In [6]:
##### train function ######

def train_WS (model, train_data, mini_batch_size=100, optimizer = optim.SGD,
                criterion = nn.CrossEntropyLoss(), n_epochs=50, eta=1e-1, lambda_l2 = 0):
    
    """
    Train network with simple weight sharing
    
    """
    # create data loader
    train_loader = DataLoader(train_data, batch_size=mini_batch_size, shuffle=True)

    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    
    for e in range(n_epochs):
        epoch_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            input_, target_, classes_ = data
            
            out_loss  = criterion(model(input_), target_)
            epoch_loss += out_loss
            
            if lambda_l2 != 0:
                for p in model.parameters():
                    epoch_loss += lambda_l2 * p.pow(2).sum() # add an l2 penalty term to the loss 
            
            optimizer.zero_grad()
            out_loss.backward()
            optimizer.step()
            
        print('Train Epoch: {}  | Loss {:.6f}'.format(
                e, epoch_loss.item()))
        
#########################################################################################################################
#########################################################################################################################

### test function  ###

def test_WS (model, test_data, mini_batch_size=100, criterion = nn.CrossEntropyLoss()):
    
    """
    Test function to calculate prediction accuracy of a binary cnn classifier
    
    """
    
    # create test laoder
    test_loader = DataLoader(test_data, batch_size=mini_batch_size, shuffle=True)
    
    model.eval()
    test_loss = 0
    nb_errors=0
    
    with torch.no_grad():
        
        for i, data in enumerate(test_loader, 0):
            input_, target_, classes_ = data
            output = model(input_) 
            batch_loss = criterion(output, target_)
            test_loss += batch_loss
            
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target_[k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
                                   
             
        print('\nTest set | Loss: {:.4f} | Accuracy: {:.0f}% | # misclassified : {}/{}\n'.format(
        test_loss.item(), 100 * (len(test_data)-nb_errors)/len(test_data), nb_errors, len(test_data)))
        
        print(nb_errors)
        
    

In [7]:
model = LeNet_WS_sequential(500)
train_WS(model, train_data)
test_WS(model, test_data)

Train Epoch: 0  | Loss 6.895623
Train Epoch: 1  | Loss 6.879736
Train Epoch: 2  | Loss 6.874956
Train Epoch: 3  | Loss 6.865646
Train Epoch: 4  | Loss 6.862055
Train Epoch: 5  | Loss 6.848652
Train Epoch: 6  | Loss 6.828834
Train Epoch: 7  | Loss 6.803328
Train Epoch: 8  | Loss 6.760359
Train Epoch: 9  | Loss 6.679596
Train Epoch: 10  | Loss 6.508476
Train Epoch: 11  | Loss 6.164664
Train Epoch: 12  | Loss 5.486394
Train Epoch: 13  | Loss 4.597522
Train Epoch: 14  | Loss 4.025372
Train Epoch: 15  | Loss 3.539963
Train Epoch: 16  | Loss 3.185930
Train Epoch: 17  | Loss 2.914316
Train Epoch: 18  | Loss 3.096124
Train Epoch: 19  | Loss 2.440158
Train Epoch: 20  | Loss 2.000538
Train Epoch: 21  | Loss 1.894639
Train Epoch: 22  | Loss 1.806071
Train Epoch: 23  | Loss 1.604826
Train Epoch: 24  | Loss 1.502590
Train Epoch: 25  | Loss 1.463696
Train Epoch: 26  | Loss 1.429723
Train Epoch: 27  | Loss 0.802154
Train Epoch: 28  | Loss 0.759072
Train Epoch: 29  | Loss 0.370543
Train Epoch: 30  | L